In [1]:
!gdown 15Jhd2ZYh8fH8T79CslRwhTi9qqr1q041

Downloading...
From: https://drive.google.com/uc?id=15Jhd2ZYh8fH8T79CslRwhTi9qqr1q041
To: /content/best.pt
100% 14.4M/14.4M [00:00<00:00, 28.4MB/s]


In [2]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output

print(f"Versi torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 15093, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 15093 (delta 0), reused 3 (delta 0), pack-reused 15088
Receiving objects: 100% (15093/15093), 13.63 MiB | 13.77 MiB/s, done.
Resolving deltas: 100% (10436/10436), done.
/content/yolov5
     |████████████████████████████████| 1.6 MB 9.6 MB/s 
     |████████████████████████████████| 41 kB 571 kB/s 
     |████████████████████████████████| 145 kB 21.3 MB/s 
     |████████████████████████████████| 178 kB 57.0 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 138 kB 57.5 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
Versi torch 1.12.1+cu113 (CPU)


In [3]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [4]:
# buat folder
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [5]:
#ambil dataset dari roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="5s6pm49rP45QR1YXLlYL")
project = rf.workspace("daniel-morantha-yt377").project("ayamdeteksi")
dataset = project.version(2).download("yolov5")


# rf = Roboflow(api_key="5s6pm49rP45QR1YXLlYL")
# project = rf.workspace("daniel-morantha-yt377").project("ayamdeteksi")
# dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/datasets/AyamDeteksi-2 in yolov5pytorch:: 100%|██████████| 492/492 [00:00<00:00, 1474.52it/s]


**Prediksi untuk 1 Gambar Test**

In [6]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', '/content/best.pt')  # custom trained model

# Images
im = '/content/datasets/AyamDeteksi-2/test/images/20221104_081444_jpg.rf.6ce7a28d5c9cadc76eac0c426b0c76a7.jpg'  # or file, Path, URL, PIL, OpenCV, numpy, list

# Inference
results = model(im)

results.pandas().xyxy[0]

deteksi_class = results.pandas().xyxy[0].sort_values('confidence',ascending=False)['name'][0]
print(deteksi_class)

You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-11-18 Python-3.7.15 torch-1.12.1+cu113 CPU

Fusing layers... 


[Errno 2] No such file or directory: '/usr/local/lib/python3.7/dist-packages/cycler-0.11.0.dist-info/METADATA'


Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


AyamSegar


**Prediksi untuk Semua File Gambar Test**

In [7]:
import glob
import os.path
file_test_images = glob.glob('/content/datasets/AyamDeteksi-2/test/images/*.jpg')

In [8]:
class_prediksi  = []
file_image_name = []

for file_image in file_test_images:
  image_name = os.path.basename(file_image).split('.')[0]
  results = model(file_image)
  results.pandas().xyxy[0]
  deteksi_class = results.pandas().xyxy[0].sort_values('confidence',ascending=False)['name'][0]

  class_prediksi.append(deteksi_class)
  file_image_name.append(image_name)

In [9]:
import pandas as pd
df_prediksi_test = pd.DataFrame({'image_name':file_image_name,
                                 'prediksi':class_prediksi})

In [10]:
df_prediksi_test

,image_name,prediksi
0,20221104_082508_jpg,AyamSegar
1,IMG_5038_jpg,AyamTiren
2,IMG_5032_jpg,AyamTiren
3,20221104_082538_jpg,AyamSegar
4,IMG_5068_jpg,AyamTiren
5,IMG_5234_JPG,AyamSegar
6,20221104_081444_jpg,AyamSegar
7,IMG_5029_jpg,AyamTiren
8,IMG_5075_jpg,AyamTiren
9,20221104_081504_jpg,AyamSegar


In [11]:
file_test_labels = glob.glob('/content/datasets/AyamDeteksi-2/test/labels/*.txt')

In [12]:
class_actual  = []
file_image_name = []

labels = ['AyamSegar','AyamTiren']

for file_label in file_test_labels:
  image_name = os.path.basename(file_label).split('.')[0]
  file_text = open(file_label, 'r')
  file_lines = file_text.readlines()
  class_actual.append(labels[int(file_lines[0].split(' ')[0])])
  file_image_name.append(image_name)

In [13]:
df_label_test = pd.DataFrame({'image_name':file_image_name,
                              'label':class_actual})

In [14]:
df_label_test

,image_name,label
0,IMG_5234_JPG,AyamSegar
1,20221104_081444_jpg,AyamSegar
2,IMG_5029_jpg,AyamTiren
3,20221104_082508_jpg,AyamSegar
4,IMG_5038_jpg,AyamTiren
5,IMG_5068_jpg,AyamTiren
6,20221104_081504_jpg,AyamSegar
7,IMG_5075_jpg,AyamTiren
8,20221104_082538_jpg,AyamSegar
9,IMG_5032_jpg,AyamTiren


In [15]:
df_prediksi_test = pd.merge(
    df_prediksi_test,
    df_label_test,
    how="inner",
    on='image_name')

In [16]:
df_prediksi_test

,image_name,prediksi,label
0,20221104_082508_jpg,AyamSegar,AyamSegar
1,IMG_5038_jpg,AyamTiren,AyamTiren
2,IMG_5032_jpg,AyamTiren,AyamTiren
3,20221104_082538_jpg,AyamSegar,AyamSegar
4,IMG_5068_jpg,AyamTiren,AyamTiren
5,IMG_5234_JPG,AyamSegar,AyamSegar
6,20221104_081444_jpg,AyamSegar,AyamSegar
7,IMG_5029_jpg,AyamTiren,AyamTiren
8,IMG_5075_jpg,AyamTiren,AyamTiren
9,20221104_081504_jpg,AyamSegar,AyamSegar


In [17]:
from sklearn.metrics import confusion_matrix
y_actual = df_prediksi_test['label']
y_pred = df_prediksi_test['prediksi']
confusion_matrix(y_actual, y_pred)

array([[5, 0],
       [0, 5]])

In [18]:
from sklearn.metrics import classification_report
print(classification_report(y_actual, y_pred))

              precision    recall  f1-score   support

   AyamSegar       1.00      1.00      1.00         5
   AyamTiren       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



#Evaluasi Data val 

In [19]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', '/content/best.pt')  # custom trained model

# Images
im = '/content/datasets/AyamDeteksi-2/valid/images/20221104_081226_jpg.rf.425359a3b74e824c5ffe5182e3b5558f.jpg'  # or file, Path, URL, PIL, OpenCV, numpy, list

# Inference
results = model(im)

results.pandas().xyxy[0]

deteksi_class = results.pandas().xyxy[0].sort_values('confidence',ascending=False)['name'][0]
print(deteksi_class)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-18 Python-3.7.15 torch-1.12.1+cu113 CPU



[Errno 2] No such file or directory: '/usr/local/lib/python3.7/dist-packages/pyparsing-3.0.9.dist-info/METADATA'


Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


AyamSegar


In [20]:
import glob
import os.path
file_test_images = glob.glob('/content/datasets/AyamDeteksi-2/valid/images/*.jpg')

In [21]:
class_prediksi  = []
file_image_name = []

for file_image in file_test_images:
  image_name = os.path.basename(file_image).split('.')[0]
  results = model(file_image)
  results.pandas().xyxy[0]
  deteksi_class = results.pandas().xyxy[0].sort_values('confidence',ascending=False)['name'][0]

  class_prediksi.append(deteksi_class)
  file_image_name.append(image_name)

In [22]:
import pandas as pd
df_prediksi_val = pd.DataFrame({'image_name':file_image_name,
                                 'prediksi':class_prediksi})

In [23]:
df_prediksi_val

,image_name,prediksi
0,20221104_082512_jpg,AyamSegar
1,20221104_082532_jpg,AyamSegar
2,IMG_4919_jpg,AyamTiren
3,20221115_133221_jpg,AyamSegar
4,20221104_081228_jpg,AyamSegar
5,20221104_081226_jpg,AyamSegar
6,IMG_5040_jpg,AyamTiren
7,20221104_081241_jpg,AyamSegar
8,IMG_5067_jpg,AyamTiren
9,20221104_082515_jpg,AyamSegar


In [24]:
file_test_labels = glob.glob('/content/datasets/AyamDeteksi-2/valid/labels/*.txt')

In [25]:
class_actual  = []
file_image_name = []

labels = ['AyamSegar','AyamTiren']

for file_label in file_test_labels:
  image_name = os.path.basename(file_label).split('.')[0]
  file_text = open(file_label, 'r')
  file_lines = file_text.readlines()
  class_actual.append(labels[int(file_lines[0].split(' ')[0])])
  file_image_name.append(image_name)

In [26]:
df_label_val = pd.DataFrame({'image_name':file_image_name,
                              'label':class_actual})

In [27]:
df_label_val

,image_name,label
0,IMG_5120_jpg,AyamTiren
1,20221104_081241_jpg,AyamSegar
2,20221104_081217_jpg,AyamSegar
3,IMG_5028_jpg,AyamTiren
4,20221104_081233_jpg,AyamSegar
5,IMG_5040_jpg,AyamTiren
6,IMG_5122_jpg,AyamTiren
7,20221104_082515_jpg,AyamSegar
8,20221104_081226_jpg,AyamSegar
9,IMG_5239_JPG,AyamSegar


In [28]:
df_prediksi_val = pd.merge(
    df_prediksi_val,
    df_label_val,
    how="inner",
    on='image_name')

In [29]:
df_prediksi_val

,image_name,prediksi,label
0,20221104_082512_jpg,AyamSegar,AyamSegar
1,20221104_082532_jpg,AyamSegar,AyamSegar
2,IMG_4919_jpg,AyamTiren,AyamTiren
3,20221115_133221_jpg,AyamSegar,AyamSegar
4,20221104_081228_jpg,AyamSegar,AyamSegar
5,20221104_081226_jpg,AyamSegar,AyamSegar
6,IMG_5040_jpg,AyamTiren,AyamTiren
7,20221104_081241_jpg,AyamSegar,AyamSegar
8,IMG_5067_jpg,AyamTiren,AyamTiren
9,20221104_082515_jpg,AyamSegar,AyamSegar


In [30]:
from sklearn.metrics import confusion_matrix
y_actual = df_prediksi_val['label']
y_pred = df_prediksi_val['prediksi']
confusion_matrix(y_actual, y_pred)

array([[11,  0],
       [ 0,  9]])

In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_actual, y_pred))

              precision    recall  f1-score   support

   AyamSegar       1.00      1.00      1.00        11
   AyamTiren       1.00      1.00      1.00         9

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

